In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import getpass

username = getpass.getuser()
pd.read_csv(f'C:/Users/{username}/SynologyDrive/AI/Short_circuit_exp_2_data/raw_data/Cr0.5/Cr0.5_1_006_DC.csv')

,Index,Test_Time(s),Cycle_No.,Cycle_Time(s),Step_No.,Step_Time(s),Current(A),Voltage(V),AuxV1(V),AuxV2(V),AuxV3(V),Temp.('C),OCP(V),Power(W),Load(Ohm),Acc.Q(Ah),|Q|(Ah),Range
0,1,0:00:00:30.000,1,0:00:00:30.000,1,0:00:00:30.000,0.000000,3.62820,0.0,0.0,0.0,0.0,3.62820,0.000000,0.00,0.000000,0.000000,101mA
1,2,0:00:01:00.000,1,0:00:01:00.000,1,0:00:01:00.000,0.000000,3.62820,0.0,0.0,0.0,0.0,3.62820,0.000000,0.00,0.000000,0.000000,101mA
2,3,0:00:01:30.000,1,0:00:01:30.000,1,0:00:01:30.000,0.000000,3.62804,0.0,0.0,0.0,0.0,3.62804,0.000000,0.00,0.000000,0.000000,101mA
3,4,0:00:02:00.000,1,0:00:02:00.000,1,0:00:02:00.000,0.000000,3.62804,0.0,0.0,0.0,0.0,3.62804,0.000000,0.00,0.000000,0.000000,101mA
4,5,0:00:02:30.000,1,0:00:02:30.000,1,0:00:02:30.000,0.000000,3.62820,0.0,0.0,0.0,0.0,3.62820,0.000000,0.00,0.000000,0.000000,101mA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439501,439502,6:00:25:51.850,15,3:01:04:37.400,5,2:23:21:16.850,0.001292,4.20073,0.0,0.0,0.0,0.0,3.61981,0.005427,3251.32,0.051869,0.048585,10mA
439502,439503,6:00:25:52.850,15,3:01:04:38.400,5,2:23:21:17.850,0.001282,4.20057,0.0,0.0,0.0,0.0,3.61981,0.005386,3276.15,0.051870,0.048585,10mA
439503,439504,6:00:25:53.850,15,3:01:04:39.400,5,2:23:21:18.850,0.001274,4.20041,0.0,0.0,0.0,0.0,3.61981,0.005353,3295.80,0.051870,0.048586,10mA
439504,439505,6:00:25:54.850,15,3:01:04:40.400,5,2:23:21:19.850,0.001262,4.20041,0.0,0.0,0.0,0.0,3.61981,0.005303,3327.11,0.051870,0.048586,10mA


In [2]:
dataset_directory=f'C:/Users/{username}/SynologyDrive/AI/Short_circuit_exp_2_data/raw_data/Cr0.5'
file_list=os.listdir(dataset_directory)
file_list=[os.path.join(dataset_directory, file) for file in file_list if file.endswith('csv')]
print(len(file_list))

new_directory_path=f'C:/Users/{username}/SynologyDrive/AI/Short_circuit_exp_2_data/pre_ver_processed_data'

6


In [3]:
cell_name=file_list[-1].split('\\')[-1].replace('_DC.csv','')[:-4]
cell_name

'Cr0.5_6'

In [5]:
from datetime import timedelta
import os
import pandas as pd

def change_sec(time):
    days, hours, minutes, seconds_milliseconds = time.split(':')
    seconds, milliseconds = seconds_milliseconds.split('.')
    time_delta = timedelta(days=int(days), hours=int(hours), minutes=int(minutes), seconds=int(seconds), milliseconds=int(milliseconds))
    total_seconds = int(days) * 86400 + int(hours) * 3600 + int(minutes) * 60 + int(seconds) + int(milliseconds) / 1000
    return time_delta.total_seconds()

def process_file(file):
    df_res = pd.read_csv(file)  # 전체 파일을 pandas 데이터프레임으로 변환
    cell_name = os.path.basename(file).replace('_DC.csv', '')[:-4]

    print(str(cell_name))
    cycle_number = df_res["Cycle_No."].unique().tolist()

    cycle_data = []

    #각 cycle 별로 processing
    for cyc in cycle_number:
        cur_df = df_res[df_res['Cycle_No.'] == cyc]   # 현재 사이클 데이터프레임 슬라이싱
        
        current_in_A = cur_df["Current(A)"].tolist()  # 현 사이클의 Current, Voltage, Time데이터 타입 리스트로 변경
        voltage_in_V = cur_df['Voltage(V)'].tolist()        
        
        timedata = cur_df['Test_Time(s)'].tolist()

        time_in_s = [change_sec(time) for time in timedata] #Time 데이터를 second 단위로 변경


        num_steps = cur_df['Step_No.'].values

        discharge_step = []
        #각 step 별로 discharge step인지 check
        for step in num_steps:
            step_df = cur_df[cur_df['Step_No.'] == step] #각 step별로 discharge step 필터링
            if step_df['Current(A)'].median() < 0:
                discharge_step.append(step)
                discharge_step.append(step + 1)

        index = [cur_df[cur_df['Step_No.'] == step_num].index.to_list() for step_num in discharge_step]

        chr_df = cur_df.copy()
        dis_df = cur_df.copy()

        index = sum(index, [])

        chr_df.loc[index, '|Q|(Ah)'] = 0
        dis_df.loc[[id for id in dis_df.index if id not in index], '|Q|(Ah)'] = 0

        discharge_capacity_in_Ah = dis_df['|Q|(Ah)'].tolist()
        charge_capacity_in_Ah = chr_df['|Q|(Ah)'].tolist()
        assert len(discharge_capacity_in_Ah) == len(charge_capacity_in_Ah), "capacity length of dis & chr is not equal "

        temperature_in_C = None
        internal_resistance_in_ohm = None
        cyc_dict = {
            'cycle_number': cyc,
            'step': num_steps,
            'time_in_s': time_in_s,
            'current_in_A': current_in_A,
            'voltage_in_V': voltage_in_V,
            'discharge_capacity_in_Ah': discharge_capacity_in_Ah,
            'charge_capacity_in_Ah': charge_capacity_in_Ah,
            'temperature_in_C': temperature_in_C,
            'internal_resistance_in_ohm': internal_resistance_in_ohm
        }
        cycle_data.append(cyc_dict)

    dataframes = {
        'cell_id': cell_name,
        'cycle_data': cycle_data,
        'form_factor': 'coin_cell',
        'anode_material': 'Li-metal',
        'cathode_material': 'NCM523',
        'electrolyte_material': None,
        'nominal_capacity_in_Ah': 0.0034,
        'depth_of_charge': 1.0,
        'depth_of_discharge': 1.0,
        'already_spent_cycles': int(cycle_number[0] - 1),
        'max_voltage_limit_in_V': 4.3,
        'min_voltage_limit_in_V': 3.0,
        'max_current_limit_in_A': None,
        'min_current_limit_in_A': None,
        'reference': None,
        'description': None,
        'charge_protocol': None,
        'discharge_protocol': None,
        'relaxation_protocol': None,
        'short_circuit_cycle': len(cycle_data)-1
    }

    new_file_name = cell_name + '.pkl'
    new_file_path = os.path.join(new_directory_path, new_file_name)
    pd.to_pickle(dataframes, new_file_path)
    print(f'Success to save \'{new_file_path}\'')

# Process each file
for file in file_list:
    process_file(file)

Cr0.5_1
step1: 3.76031231880188
